# Import Models as Python Functions

A useful feature of yggdrasil is that you can use the mechanics to call models as functions from inside Python even if the model is not written in Python. For example, the model displayed below is written in Fortran, but can be called from Python via the yggdrasil `import_as_function` method.

In [ ]:
from yggdrasil import tools
tools.display_source('models/light_v0.f90', number_lines=True)
tools.display_source('yamls/light_v0_fortran.yml', number_lines=True)

When import_as_function is called, the model yaml is loaded and models contained in the yaml are run on forked processes after being copiled as necessary. 

In [ ]:
from yggdrasil import import_as_function
light = import_as_function('yamls/light_v0_fortran.yml')

The returned function's inputs & outputs are determined by the unpaired inputs/outputs located in the yaml. In this example, the only inputs & outputs come from the server.

In [ ]:
light.model_info()

The light model can then be called as a function by providing input arguments. If the values provided do not have any units, the default units expected by the light model are assumed (i.e. days and cm).

In [ ]:
print(light(100.0, 100.0))
print(light(1.0, 2.9))
print(light(2.0, 3.0))

If units are added, the values will be converted into the units expected by the model.

In [ ]:
from yggdrasil import units
print(light(units.add_units(24.0, 'hrs'), units.add_units(2.9, 'cm')))
print(light(units.add_units(1.0, 'days'), units.add_units(0.029, 'm')))

In [ ]:
light.stop()

# Overhead
Although yggdrasil makes it easy to connect models, the connections come with a performance overhead and can never match the performance from passing messages directly within a language. For example, the three cells below make the same calculation (sine), but the first calls a Python method direction, the second calls the same Python method but via yggdrasil, and the third calls the Fortran `SIN` function via yggdrasil. 

The direct call to the Python method (first cell) is much more efficient. Similarly, if two models are written in the same language, it will be much more preformant to call the code directly (unless the communication pattern allows models to make computations in parallel).

In [ ]:
# Calculation using numpy in Python
import numpy as np
%timeit np.sin(0.0)

In [ ]:
# Calculation calling numpy in Python via yggdrasil
sine_python = import_as_function('yamls/sine_model_python.yml')
%timeit sine_python(0.0)
sine_python.stop()

In [ ]:
# Calculation calling the fortran SIN function via yggdrasil
sine_fortran = import_as_function('yamls/sine_model_fortran.yml')
%timeit sine_fortran(0.0)
sine_fortran.stop()

# Models built by make/cmake
Many of the existing computational models contain a large amount of source code organized in directories, modules, etc. For models written in compiled languages, this size and complexity means it is often necessary to use a build tool to compile the model. Yggdrasil allows for such cases by supporting `make` and `cmake` model language parameters.

For example, the cell below displays three YAML files. The first is the YAML required to run a C version of the `timesync` enabled roots model from the previous notebook. The second is the YAML required to run the same code, but compiled via make. The third uses cmake. 

Here yggdrasil assumes the standard build file name (`Makefile` and `CMakeLists.txt`), but the name can also be passed explicitly. Yggdrasil adds the compilation and linking flags to the build by either adjusting the environment variables used by most builds (as in the case of `make`) or creating a dummy version of the build file that adds these flags to the build rules for the selected target (as in the cas of `cmake`).

In [ ]:
tools.display_source('yamls/roots_v1_c.yml', number_lines=True)
tools.display_source_diff('yamls/roots_v1_c.yml', 'yamls/roots_v1_make.yml', number_lines=True)
tools.display_source_diff('yamls/roots_v1_c.yml', 'yamls/roots_v1_cmake.yml', number_lines=True)